## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [7]:
using DataFrames
using CSV
using Pkg; Pkg.add("ProgressBars")
using ProgressBars

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed ProgressBars ─ v1.4.0
    Updating `~/tornado_cash_anonymity_tool/Project.toml`
  [49802e3a] + ProgressBars v1.4.0
    Updating `~/tornado_cash_anonymity_tool/Manifest.toml`
  [49802e3a] + ProgressBars v1.4.0
Precompiling project...
  ✓ ProgressBars
1 dependency successfully precompiled in 3 seconds (176 already precompiled)


In [3]:
withdraw_transactions_df = CSV.read("../data/complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/complete_deposit_txs.csv", DataFrame);

In [4]:
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

### Function summary: filter_by_unique_gas_price

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions.

In [5]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price. 
function filter_by_unique_gas_price(transactions_df)
    unique_gas_prices = filter(row -> row.count==1, combine(groupby(transactions_df, :gas_price), nrow => :count))[!, "gas_price"]
    filter(row -> row.gas_price ∈ unique_gas_prices, transactions_df)
end

filter_by_unique_gas_price (generic function with 1 method)

In [6]:
unique_gasp_deposits = filter_by_unique_gas_price(deposit_transactions_df)
first(unique_gasp_deposits, 5)

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,String,Int64,Int64,String,String,Int128,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String31,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb3150e001eca78166fd6d66a5fd253e,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
2,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0,1200000,71302125000,0xb214faa52d8f3e8b9934e70c0fbf5bb3cf355027815a11191fb5e2f1d3f6f84beb2a7c35,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
3,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1000000000000000000,275000,595000000000,0xb214faa500fe7a849e7374f033604417bf6ccd369ea65e027f5327f58fcb8667cb469769,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
4,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10000000000000000000,1200000,5231046931,0xb214faa52456540e731a7caef51cd40086cc0236eb70b69160587d15e753084dbb4df3c5,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
5,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,57151624549,0xb214faa517409b6dc9fc6d2da0fd00fd2ea324a61922aee92d412c3e1bf394b19d91ffa9,9371665,978691,missing,missing,1,2020-10-16 09:18:01 UTC,11066064,0xfa752cf56aaec471375e2996736d20444b146199e2c3b84851cb16c65695cd3f,missing,missing,missing,57151624549,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc


### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(false, nothing)$ when such a deposit is not found.

In [21]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.
function same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dataframe)
    for row in eachrow(unique_gas_price_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic (generic function with 1 method)

In [20]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.
function same_gas_price_heuristic_by_pool(withdrawal_transaction, unique_gas_price_deposit_dataframe)
    for row in eachrow(unique_gas_price_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp) &&
            (withdrawal_transaction.tornado_cash_address == row.tornado_cash_address)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic_by_pool (generic function with 1 method)

In [9]:
same_gas_price_heuristic(withdraw_transactions_df[1,:], unique_gasp_deposits)

(false, nothing)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [17]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.
function apply_same_gas_price_heuristic(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price(deposit_dataframe)
    withdrawal_to_deposit = []
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe))
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            push!(withdrawal_to_deposit, (same_gas_deposit_hash[2], withdraw_row.hash))
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic (generic function with 1 method)

In [18]:
@time link_same_gas = apply_same_gas_price_heuristic(deposit_transactions_df, withdraw_transactions_df);

0.0%┣                                        ┫ 0/83.8k [00:00<-1:-9:-49, -0s/it]
0.0%┣                                        ┫ 1/83.8k [00:00<Inf:Inf, InfGs/it]
0.0%┣                                          ┫ 26/83.8k [00:00<07:11, 194it/s]
0.1%┣                                          ┫ 49/83.8k [00:00<05:15, 266it/s]
0.1%┣                                          ┫ 63/83.8k [00:00<05:12, 268it/s]
0.1%┣                                          ┫ 82/83.8k [00:00<04:52, 287it/s]
0.1%┣                                          ┫ 99/83.8k [00:00<04:45, 294it/s]
0.1%┣                                         ┫ 122/83.8k [00:00<04:26, 315it/s]
0.2%┣                                         ┫ 139/83.8k [00:00<04:24, 317it/s]
0.2%┣                                         ┫ 162/83.8k [00:00<04:14, 330it/s]
0.2%┣                                         ┫ 175/83.8k [00:01<04:19, 322it/s]
0.2%┣                                         ┫ 179/83.8k [00:01<04:43, 295it/s]
0.2%┣                       

2.7%┣█                                       ┫ 2.3k/83.8k [00:05<03:16, 416it/s]
2.7%┣█                                       ┫ 2.3k/83.8k [00:06<03:17, 413it/s]
2.8%┣█                                       ┫ 2.3k/83.8k [00:06<03:16, 415it/s]
2.8%┣█▏                                      ┫ 2.3k/83.8k [00:06<03:16, 416it/s]
2.8%┣█▏                                      ┫ 2.4k/83.8k [00:06<03:15, 417it/s]
2.9%┣█▏                                      ┫ 2.4k/83.8k [00:06<03:15, 417it/s]
2.9%┣█▏                                      ┫ 2.4k/83.8k [00:06<03:14, 419it/s]
2.9%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:15, 418it/s]
3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:14, 419it/s]
3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:14, 420it/s]
3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:13, 421it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:13, 422it/s]
3.1%┣█▎                     

5.6%┣██▎                                     ┫ 4.7k/83.8k [00:11<03:05, 428it/s]
5.6%┣██▎                                     ┫ 4.7k/83.8k [00:11<03:05, 427it/s]
5.6%┣██▎                                     ┫ 4.7k/83.8k [00:11<03:05, 428it/s]
5.7%┣██▎                                     ┫ 4.7k/83.8k [00:11<03:05, 428it/s]
5.7%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:04, 429it/s]
5.7%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:04, 430it/s]
5.8%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:03, 430it/s]
5.8%┣██▎                                     ┫ 4.9k/83.8k [00:11<03:03, 431it/s]
5.8%┣██▎                                     ┫ 4.9k/83.8k [00:11<03:04, 430it/s]
5.8%┣██▍                                     ┫ 4.9k/83.8k [00:11<03:03, 430it/s]
5.9%┣██▍                                     ┫ 4.9k/83.8k [00:11<03:03, 431it/s]
5.9%┣██▍                                     ┫ 5.0k/83.8k [00:11<03:03, 431it/s]
5.9%┣██▍                    

8.3%┣███▍                                    ┫ 7.0k/83.8k [00:16<03:00, 426it/s]
8.4%┣███▍                                    ┫ 7.0k/83.8k [00:16<03:01, 425it/s]
8.4%┣███▍                                    ┫ 7.0k/83.8k [00:17<03:01, 425it/s]
8.4%┣███▍                                    ┫ 7.0k/83.8k [00:17<03:01, 425it/s]
8.4%┣███▍                                    ┫ 7.1k/83.8k [00:17<03:01, 425it/s]
8.5%┣███▍                                    ┫ 7.1k/83.8k [00:17<03:01, 424it/s]
8.5%┣███▍                                    ┫ 7.1k/83.8k [00:17<03:01, 423it/s]
8.5%┣███▍                                    ┫ 7.1k/83.8k [00:17<03:01, 423it/s]
8.5%┣███▍                                    ┫ 7.1k/83.8k [00:17<03:01, 423it/s]
8.5%┣███▍                                    ┫ 7.2k/83.8k [00:17<03:01, 423it/s]
8.6%┣███▍                                    ┫ 7.2k/83.8k [00:17<03:01, 423it/s]
8.6%┣███▍                                    ┫ 7.2k/83.8k [00:17<03:01, 423it/s]
8.6%┣███▌                   

11.0%┣████▎                                  ┫ 9.2k/83.8k [00:22<02:58, 419it/s]
11.0%┣████▎                                  ┫ 9.2k/83.8k [00:22<02:58, 420it/s]
11.1%┣████▎                                  ┫ 9.3k/83.8k [00:22<02:58, 420it/s]
11.1%┣████▎                                  ┫ 9.3k/83.8k [00:22<02:57, 420it/s]
11.1%┣████▍                                  ┫ 9.3k/83.8k [00:22<02:57, 420it/s]
11.1%┣████▍                                  ┫ 9.3k/83.8k [00:22<02:58, 420it/s]
11.2%┣████▍                                  ┫ 9.3k/83.8k [00:22<02:57, 420it/s]
11.2%┣████▍                                  ┫ 9.4k/83.8k [00:22<02:57, 420it/s]
11.2%┣████▍                                  ┫ 9.4k/83.8k [00:22<02:57, 421it/s]
11.3%┣████▍                                  ┫ 9.4k/83.8k [00:22<02:57, 421it/s]
11.3%┣████▍                                  ┫ 9.5k/83.8k [00:22<02:56, 421it/s]
11.3%┣████▍                                  ┫ 9.5k/83.8k [00:23<02:57, 421it/s]
11.4%┣████▍                 

...

92.1%┣███████████████████████████████████   ┫ 77.2k/83.8k [02:55<00:15, 441it/s]
92.2%┣███████████████████████████████████   ┫ 77.2k/83.8k [02:55<00:15, 440it/s]
92.2%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:55<00:15, 441it/s]
92.2%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:55<00:15, 441it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:55<00:15, 441it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:56<00:15, 440it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:56<00:15, 440it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [02:56<00:15, 440it/s]
92.3%┣███████████████████████████████████   ┫ 77.4k/83.8k [02:56<00:15, 440it/s]
92.4%┣███████████████████████████████████   ┫ 77.4k/83.8k [02:56<00:15, 440it/s]
92.4%┣███████████████████████████████████   ┫ 77.4k/83.8k [02:56<00:15, 440it/s]
92.4%┣███████████████████████████████████   ┫ 77.4k/83.8k [02:56<00:14, 440it/s]
92.4%┣██████████████████████

95.4%┣████████████████████████████████████▎ ┫ 79.9k/83.8k [03:01<00:09, 443it/s]
95.4%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:01<00:09, 443it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:01<00:09, 443it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:01<00:08, 443it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:01<00:08, 443it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:01<00:08, 443it/s]
95.6%┣████████████████████████████████████▍ ┫ 80.1k/83.8k [03:01<00:08, 443it/s]
95.6%┣████████████████████████████████████▍ ┫ 80.1k/83.8k [03:01<00:08, 443it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [03:01<00:08, 443it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [03:01<00:08, 443it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [03:01<00:08, 443it/s]
95.8%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [03:01<00:08, 443it/s]
95.8%┣██████████████████████

98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:06<00:02, 445it/s]
98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:06<00:02, 445it/s]
99.0%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:06<00:02, 445it/s]
99.0%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:06<00:02, 446it/s]
99.0%┣█████████████████████████████████████▋┫ 83.0k/83.8k [03:06<00:02, 446it/s]
99.1%┣█████████████████████████████████████▋┫ 83.0k/83.8k [03:06<00:02, 446it/s]
99.1%┣█████████████████████████████████████▋┫ 83.0k/83.8k [03:06<00:02, 446it/s]
99.1%┣█████████████████████████████████████▋┫ 83.0k/83.8k [03:06<00:02, 446it/s]
99.2%┣█████████████████████████████████████▊┫ 83.1k/83.8k [03:06<00:02, 446it/s]
99.2%┣█████████████████████████████████████▊┫ 83.1k/83.8k [03:06<00:02, 446it/s]
99.2%┣█████████████████████████████████████▊┫ 83.1k/83.8k [03:07<00:01, 446it/s]
99.3%┣█████████████████████████████████████▊┫ 83.2k/83.8k [03:07<00:01, 446it/s]
99.3%┣██████████████████████

188.340891 seconds (4.92 G allocations: 73.394 GiB, 9.13% gc time, 0.18% compilation time)


100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [03:08<00:00, 446it/s]
100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [03:08<00:00, 446it/s]


In [19]:
# A DataFrame with the linked transactions.
linked_transactions_df = DataFrame("deposit_hash"=>[tuple[1] for tuple in link_same_gas],"withdrawl_hash"=>[tuple[2] for tuple in link_same_gas])

,deposit_hash,withdrawl_hash
,String,String
1,0x84508c35b90983284f9605117333ed9c379be64bf82f3c04fd7338a1a3b5dc62,0x6b1ffed49640dce414b1110fb8111c6e6d5cae0707db526b4628d1088a0ae0be
2,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0x0f742f92162f7a30c07d314a891acebae3baf229a667ae571dd501d25fa61528
3,0x3156e68753b1d12cd68da86a45f1f2b96b48c73c3d982b193c75b4cee0e7a44c,0xf6b761a8d27dddf0710b4cc2c0c5f8a1d740b5500cd62021c7f91dba05aba966
4,0x8c72a5f48918f1c6a79ce219e042ddb81f1a2442138a99c7272d4856bf58246c,0xc5205147de3a60cb321c005e59a8ea0004bcccb8389b9a48540b628ab4767f77
5,0x6a082db2993c2beaa1134cd3f266d9b79b0cfad43dbc635b4488ab64106ab452,0xc40d7044612eae0ab3986df553b0043374516332f53581480c2f9570c2faee2e
6,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0x60bd987f24c03d889efeb6c0adc7ccea325d9237b8890f66623732515743c997
7,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0xa0e14aaed7998d8c3ea94d522d7a1f7ca13891ad2195f72b74648245db0c6b83
8,0x4cb1619f2d51cc7ecb926905747804e589765634ea737d957a19383a7c32162f,0x212b85e840b1789d00e4ea41ff2317e146ec1fe7df6994b3c323f507ef549f9b
9,0x3b81998758b17d407c129307c8f2ab0fd5066de0d6c9432be81602257662f4cf,0x208d2874e7130b2e1428ad1b688d3562448606257cfd4e30785b4d5275372eef
